In [1]:
#수학 체인
#사용자 질문 >> 수학 함수 호출 체인

!pip install langchain
!pip install openai
!pip install langchain_openai


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.0/990.0 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.5/373.5 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 11.7 MB/s eta 0:00:00


In [2]:
import os
import openai

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get('open_ai_key_team2')

In [4]:
#수학공식 푸는 패키지 numexpr
import numexpr as ne


In [5]:
ne.evaluate('15**0.314')

array(2.34041383)

In [6]:
#내가 원하는 건
#자연어 질문 >> 수학공식 생성 >> numexpr >> 답변

from langchain.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.prompts.chat import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate, AIMessagePromptTemplate

# PromptTemplate : 대화의 전반적인 흐름 제어하는 템플릿
# SystemMessagePromptTemplate : 시스템 메시지만 포함하는 템플릿
# HumanMessagePromptTemplate: 사용자 메시지만 포함하는 템플릿

In [7]:
# numexpr.evaluate("수학공식")
# 자연어 >> llm >> 수학공식 >> parsing >> numexpr.evalute
# 출력형식
'''
```text
13**0.3432
```
'''
# ai 가 정해진 질문 이해, 답변 role (역할) 부여

system_content_template = """You are a helpful assistant that make an only ARGUMENT for numexpr.evaluate according to user query such as
```text
13**0.3432
```
when user query is 'What is 13 raised to the .3432 power?'
Please response as code block using text back quote like
```text
13**0.3432
```
without other description."""

In [8]:
#실제 사용자가 어떤 형식으로 input 전달할지 정의

human_content_template = "{user_input}"

#템플릿 정의
chat_prompt_template = ChatPromptTemplate.from_messages([
                        ("system", system_content_template),
                        ("human", human_content_template)
                    ])

chat_model = ChatOpenAI(temperature=0)

chain = chat_prompt_template | chat_model
chain.invoke({"user_input" : "What is 15 raised to the .3432 power?"})

AIMessage(content='```text\n15**0.3432\n```', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 102, 'total_tokens': 113}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-717d53d1-459a-4d26-815e-11bf4db57889-0', usage_metadata={'input_tokens': 102, 'output_tokens': 11, 'total_tokens': 113})

In [9]:
#모델 출력을 특정 형식으로 구문 분석 >> 필요에 따라 추가적인 처리를 수행하는 데 사용
from langchain.schema import BaseOutputParser

# ```text\n15**0.3432\n``` >> "15**0.3432"

class CodeOutPutParser(BaseOutputParser):
    def parse(self, text: str):
        # text 변수 문자열(str) 전달되어야함
        value = text.strip().split("```") # 공백 제거, ```기준 분리(구분)
        print(value)
        return value[1].split("\n")[1]

chain = chat_prompt_template | chat_model | CodeOutPutParser()

output = chain.invoke({'user_input' : "What is 15 raised to the .3432 power?"})
print(output)


['', 'text\n15**0.3432\n', '']
15**0.3432


In [10]:
chain = chat_prompt_template | chat_model | CodeOutPutParser() | ne.evaluate

output = chain.invoke({'user_input' : "What is 15 raised to the .3432 power?"})
print(output)

['', 'text\n15**0.3432\n', '']
2.5329960941138205


In [11]:
from langchain_core.runnables import RunnablePassthrough, RunnableParallel, RunnableLambda

# chain.invoke({'user_input' : "What is 15 raised to the .3432 power?"}) >> 이 부분 {'user_input' : RunnablePassthrough()}
chain = ({'user_input' : RunnablePassthrough()} | chat_prompt_template | chat_model | CodeOutPutParser() | ne.evaluate)

output = chain.invoke({'user_input' : "What is 15 raised to the .3432 power?"})
print(output)

['', 'text\n15**0.3432\n', '']
2.5329960941138205
